# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,praia da vitoria,38.7333,-27.0667,21.14,83,20,5.11,PT,1717764499
1,1,juneau,58.3019,-134.4197,7.00,100,75,0.00,US,1717764286
2,2,el gouna,27.3942,33.6782,40.05,35,0,6.97,EG,1717764501
3,3,sault ste. marie,46.5168,-84.3333,11.31,96,100,5.14,CA,1717764485
4,4,hawaiian paradise park,19.5933,-154.9731,21.73,95,40,1.54,US,1717764502


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
	"Lng", 
	"Lat", 
	geo=True,
	tiles="OSM",
	size="Humidity",
    frame_width = 700,
    frame_height = 500,
	color="City",
	alpha=0.5
)

# Display the map
map_plot_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
raw_city_data_df = city_data_df
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna(how='any')

# Display sample data
city_data_df.sample(n=10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
201,201,brewton,31.1052,-87.0722,26.46,94,0,1.51,US,1717764497
318,318,campina verde,-19.5356,-49.4864,26.57,40,0,3.37,BR,1717764691
161,161,la couronne,45.6113,0.0995,23.82,54,0,1.54,FR,1717764596
26,26,mesquite,32.7668,-96.5992,25.96,84,0,0.89,US,1717764198
299,299,sao joao da barra,-21.6403,-41.0511,23.05,80,0,2.26,BR,1717764681
316,316,aragarcas,-15.8975,-52.2508,23.36,73,0,0.00,BR,1717764690
485,485,north platte,41.1239,-100.7654,21.73,67,0,2.24,US,1717764790
374,374,tucumcari,35.1717,-103.7250,21.91,60,0,4.12,US,1717764723
515,515,extrema,-22.8547,-46.3183,22.29,49,0,0.40,BR,1717764809
214,214,colonia,50.9333,6.9500,22.75,43,0,2.57,DE,1717764523


In [5]:
city_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 23 entries, 8 to 602
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     23 non-null     int64  
 1   City        23 non-null     object 
 2   Lat         23 non-null     float64
 3   Lng         23 non-null     float64
 4   Max Temp    23 non-null     float64
 5   Humidity    23 non-null     int64  
 6   Cloudiness  23 non-null     int64  
 7   Wind Speed  23 non-null     float64
 8   Country     23 non-null     object 
 9   Date        23 non-null     int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 2.0+ KB


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy(deep=False)

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,fort bragg,US,35.1390,-79.0060,88,
26,mesquite,US,32.7668,-96.5992,84,
145,nova vicosa,BR,-17.8919,-39.3719,68,
161,la couronne,FR,45.6113,0.0995,54,
201,brewton,US,31.1052,-87.0722,94,
214,colonia,DE,50.9333,6.9500,43,
225,tura,IN,25.5198,90.2201,78,
276,dothan,US,31.2232,-85.3905,93,
278,haiku-pauwela,US,20.9219,-156.3051,85,
299,sao joao da barra,BR,-21.6403,-41.0511,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories":"accommodation.hotel",
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    coords = (row['Lng'], row["Lat"])
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{coords[0]},{coords[1]},{radius}"
    params["bias"] = f"proximity:{coords[0]},{coords[1]}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
fort bragg - nearest hotel: Airborne Inn Lodging
mesquite - nearest hotel: Holiday Inn Express & Suites
nova vicosa - nearest hotel: Pousada Pontal da Barra
la couronne - nearest hotel: Hôtel Marjolaine
brewton - nearest hotel: Days Inn & Suites by Wyndham Brewton
colonia - nearest hotel: Wasserturm Hotel Cologne
tura - nearest hotel: Hotel Polo Orchid
dothan - nearest hotel: GuestHouse Dothan
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
sao joao da barra - nearest hotel: Pousada Mediterrâneo
aragarcas - nearest hotel: Hotel Portal da Praia
campina verde - nearest hotel: No hotel found
namibe - nearest hotel: Nelsal Pensao
payakumbuh - nearest hotel: Hotel Mangkuto
tucumcari - nearest hotel: Pow Wow Inn
dakhla - nearest hotel: Hotel El Massira
kingman - nearest hotel: Home2 Suites by Hilton Kingman
sao vicente - nearest hotel: Pousada Vitória
warrensburg - nearest hotel: Days Inn by Wyndham Warrensburg
north platte - nearest hotel: Howard Johnson Inn - 

,City,Country,Lat,Lng,Humidity,Hotel Name
8,fort bragg,US,35.1390,-79.0060,88,Airborne Inn Lodging
26,mesquite,US,32.7668,-96.5992,84,Holiday Inn Express & Suites
145,nova vicosa,BR,-17.8919,-39.3719,68,Pousada Pontal da Barra
161,la couronne,FR,45.6113,0.0995,54,Hôtel Marjolaine
201,brewton,US,31.1052,-87.0722,94,Days Inn & Suites by Wyndham Brewton
214,colonia,DE,50.9333,6.9500,43,Wasserturm Hotel Cologne
225,tura,IN,25.5198,90.2201,78,Hotel Polo Orchid
276,dothan,US,31.2232,-85.3905,93,GuestHouse Dothan
278,haiku-pauwela,US,20.9219,-156.3051,85,Inn At Mama's Fish House
299,sao joao da barra,BR,-21.6403,-41.0511,80,Pousada Mediterrâneo


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
	"Lng", 
	"Lat", 
	geo=True,
	tiles="OSM",
	size="Humidity",
    frame_width = 700,
    frame_height = 500,
	color="City",
	alpha=0.5,
	hover_cols=["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"]
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)